In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import gridspec
from scipy import stats

from one.api import ONE
from brainbox.io.one import SessionLoader

from iblnm.util import EVENT2COLOR, _load_event_times
from iblnm.resp import get_responses

In [3]:
one = ONE()

In [4]:
# eid = '0d118e83-1450-4382-9125-5fabc5c31b88'

In [5]:
# eids = list(one.search(subject='ZFM-08757', datasets=DSETS))
# len(eids)
# eid = eids[-1]

In [6]:
df_sessions = pd.read_parquet('metadata/sessions.pqt').query('remote_photometry == True')
df_sessions = df_sessions[df_sessions['target'].apply(len) > 0]
df_sessions['target_NM'] = df_sessions.apply(lambda x: '-'.join([x['target'][0], x['NM']]), axis='columns')
df_sessions['target_NM'].value_counts()

target_NM
NBM-ACh    71
VTA-DA     37
LC-NE      37
SI-ACh     18
Name: count, dtype: int64

In [7]:
# Pick a target
target_NM = 'NBM-ACh'
df_target = df_sessions.query('target_NM == @target_NM')
df_target.groupby(['subject', 'session_type']).count()['session_n']

subject    session_type
CQ001      habituation      1
           training        17
ZFM-08751  habituation      1
           training        19
ZFM-08757  habituation      1
           training        18
ZFM-08813  habituation      3
           training         4
ZFM-08814  habituation      3
           training         4
Name: session_n, dtype: int64

In [44]:
# Pick a session
subject = 'ZFM-08814'
session_type = 'training'
session_n = -2
session = df_target.query('(subject == @subject) & (session_type == @session_type)').iloc[session_n]
eid = session['eid']
session[['subject', 'target_NM', 'session_type', 'session_n', 'start_time', 'eid']]

subject                                    ZFM-08814
target_NM                                    NBM-ACh
session_type                                training
session_n                                          6
start_time                2025-05-22T11:34:33.591190
eid             6b7c1072-6f45-4ff4-9507-e7cf53f4a7ac
Name: 2708, dtype: object

In [45]:
# Get photometry data
signals = one.load_dataset(id=eid, dataset='photometry.signal.pqt')
locations = one.load_dataset(id=eid, dataset='photometryROI.locations.pqt').reset_index()
rois = locations['ROI'].to_list()
photometry = signals[list(rois) + ['name']].set_index(signals['times']).dropna()
print(locations)

(S3) /home/crombie/Downloads/ONE/alyx.internationalbrainlab.org/mainenlab/Subjects/ZFM-08814/2025-05-22/001/alf/photometry/photometry.signal.pqt: 100%|█| 11.8M/11.8M


  ROI    fiber brain_region
0  G0  fiber00          NBM


In [46]:
# Pick an ROI
# roi = 'G0'
roi = locations['ROI'].iloc[0]

In [47]:
# Restrict raw photometry to the task period
buffer = 15
loader = SessionLoader(one, eid=eid)
## FIXME: appropriately handle cases with multiple task collections
loader.load_trials(collection='alf/task_00')
timings = [col for col in loader.trials.columns if col.endswith('_times')]
t0 = loader.trials[timings].min().min()
t1 = loader.trials[timings].max().max()
i0 = photometry.index.searchsorted(t0 - buffer)
i1 = photometry.index.searchsorted(t1 + buffer)
photometry = photometry.iloc[i0:i1].copy()

# Pull channels out of df
gcamp = photometry.query('name == "GCaMP"')
iso = photometry.query('name == "Isosbestic"')

(S3) /home/crombie/Downloads/ONE/alyx.internationalbrainlab.org/mainenlab/Subjects/ZFM-08814/2025-05-22/001/alf/task_00/_ibl_trials.stimOffTrigger_times.npy: 100%|█|
(S3) /home/crombie/Downloads/ONE/alyx.internationalbrainlab.org/mainenlab/Subjects/ZFM-08814/2025-05-22/001/alf/task_00/_ibl_trials.table.pqt: 100%|█| 34.1k/34.1k [0
(S3) /home/crombie/Downloads/ONE/alyx.internationalbrainlab.org/mainenlab/Subjects/ZFM-08814/2025-05-22/001/alf/task_00/_ibl_trials.goCueTrigger_times.npy: 100%|█| 3
(S3) /home/crombie/Downloads/ONE/alyx.internationalbrainlab.org/mainenlab/Subjects/ZFM-08814/2025-05-22/001/alf/task_00/_ibl_trials.stimOnTrigger_times.npy: 100%|█| 
(S3) /home/crombie/Downloads/ONE/alyx.internationalbrainlab.org/mainenlab/Subjects/ZFM-08814/2025-05-22/001/alf/task_00/_ibl_trials.repNum.npy: 100%|█| 3.24k/3.24k [
(S3) /home/crombie/Downloads/ONE/alyx.internationalbrainlab.org/mainenlab/Subjects/ZFM-08814/2025-05-22/001/alf/task_00/_ibl_trials.stimOff_times.npy: 100%|█| 3.24k/
(S3)

In [48]:
# Get responses to task events
session = _load_event_times(session, one)

events = ['cue', 'movement', 'reward', 'omission']
psths = []
for event in events:
    responses, tpts = get_responses(gcamp[roi], session[f'{event}_times'])
    psths.append(responses)

In [49]:
%matplotlib tk

fig = plt.figure(figsize=(12, 8))
grid = gridspec.GridSpec(2, 3)

ax2 = fig.add_subplot(grid[1, 0])
ax3 = fig.add_subplot(grid[1, 1])
ax4 = fig.add_subplot(grid[1, 2])
ax1 = fig.add_subplot(grid[0, :])

ax1.plot(gcamp[roi])
ax1.plot(iso[roi], color='gray')
ax1.set_ylabel('Signal (a.u.)')
ax1.set_title(f"{session['subject']}, {session['session_type']}, session {session['session_n']}")

colors = [EVENT2COLOR[event] for event in events]
for event, responses, color, ax in zip(events, psths, colors, [ax2, ax3, ax4, ax4]):
    ax.plot(tpts, responses.mean(axis=0), color=color, label=event)
    ax.plot(tpts, responses.mean(axis=0) - stats.sem(responses, axis=0), ls='--', color=color)
    ax.plot(tpts, responses.mean(axis=0) + stats.sem(responses, axis=0), ls='--', color=color)
    ax.axvline(0, ls='--', color='black', alpha=0.5)
    ax.axhline(0, ls='--', color='gray', alpha=0.5)
    ax.set_xlabel('Time (s)')
    if event != 'omission':
        ax.set_title(event.capitalize())
    ax.ticklabel_format(axis='y', style='sci', scilimits=[-2, 2])